In [1]:
import pathlib

import torch
import torch.nn.functional as F
from torch.nn import Linear, Module
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Network(Module):
    def __init__(self):
        super().__init__()

        self.fc_1 = Linear(10, 20)
        self.fc_2 = Linear(20, 30)
        self.fc_3 = Linear(30, 2)


    def forward(self, x):
        x = self.fc_1(x)
        x = self.fc_2(x)
        x = self.fc_3(x)

        x = F.relu(x)

        return x

### Hook :  You can register a hook on a Tensor or a nn.Module. A hook is basically a function that is executed when the either forward or backward is called.

PyTorch provides two types of hooks.

    The Forward Hook
    The Backward Hook

A forward hook is executed during the forward pass, while the backward hook is , well, you guessed it, executed when the backward function is called. Time to remind you again, these are the forward and backward functions of an Autograd.Function object.

for forward hook: hook(module, input, output) -> None \\
for backward hook:  hook(module, grad_input, grad_output) -> Tensor or None  \\

grad_input is the gradient of the input of nn.Module object w.r.t to the loss . grad_output is the gradient of the output of the nn.Module object w.r.t to the loss. 

In [6]:
if __name__ == "__main__":
    log_dir = pathlib.Path.cwd() / "tensorboard_logs"
    writer = SummaryWriter(log_dir)

    x = torch.rand(1, 10)
    net = Network()

    def activation_hook(inst, inp, out):
        """Run activation hook.
        Parameters
        ----------
        inst : torch.nn.Module
            The layer we want to attach the hook to.
        inp : tuple of torch.Tensor
            The input to the `forward` method.
        out : torch.Tensor
            The output of the `forward` method.
        """
        print(repr(inst))
        print(out)
        #writer.add_histogram(repr(inst), out)

    handle_1 = net.fc_1.register_forward_hook(activation_hook)
    # net.fc_2.register_forward_hook(activation_hook)
    # net.fc_3.register_forward_hook(activation_hook)

    #y = net(x)
    #handle_1.remove()
    y = net(x)

Linear(in_features=10, out_features=20, bias=True)
tensor([[ 0.2694,  0.0410,  0.3110,  0.7013, -0.2220,  0.2695, -0.2852,  0.2360,
         -0.6568,  0.7898,  0.3506, -0.4706,  0.5300,  0.4436, -0.2577,  0.2203,
         -0.5060,  0.5891, -0.4182, -0.5218]], grad_fn=<AddmmBackward0>)


In [ ]:
# !tensorboard --logdir=tensorboard_logs/

In [7]:
import torch 
import torch.nn as nn

class myNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv = nn.Conv2d(3,10,2, stride = 2)
    self.relu = nn.ReLU()
    self.flatten = lambda x: x.view(-1)
    self.fc1 = nn.Linear(160,5)
   
  
  def forward(self, x):
    x = self.relu(self.conv(x))
    return self.fc1(self.flatten(x))
  

net = myNet()

def hook_fn(m, i, o):
  print(m)
  print("------------Input Grad------------")

  for grad in i:
    try:
      print(grad)
    except AttributeError: 
      print ("None found for Gradient")

  print("------------Output Grad------------")
  for grad in o:  
    try:
      print(grad)
    except AttributeError: 
      print ("None found for Gradient")
  print("\n")

  
net.conv.register_backward_hook(hook_fn)
net.fc1.register_backward_hook(hook_fn)
inp = torch.randn(1,3,8,8)
out = net(inp)

(1 - out.mean()).backward()

Linear(in_features=160, out_features=5, bias=True)
------------Input Grad------------
tensor([-0.2000, -0.2000, -0.2000, -0.2000, -0.2000])
tensor([-0.2000, -0.2000, -0.2000, -0.2000, -0.2000])
------------Output Grad------------
tensor([-0.2000, -0.2000, -0.2000, -0.2000, -0.2000])


Conv2d(3, 10, kernel_size=(2, 2), stride=(2, 2))
------------Input Grad------------
None
tensor([[[[-0.0516,  0.0196],
          [ 0.0454, -0.0095]],

         [[ 0.0678, -0.0408],
          [-0.0185, -0.0377]],

         [[ 0.0136,  0.0045],
          [ 0.0091, -0.0008]]],


        [[[ 0.0232, -0.0197],
          [-0.0082,  0.1146]],

         [[-0.0988, -0.0600],
          [ 0.0197, -0.0305]],

         [[-0.0744,  0.0460],
          [-0.0404,  0.0729]]],


        [[[ 0.0126, -0.0383],
          [-0.0209,  0.0138]],

         [[-0.0587,  0.0298],
          [ 0.0336,  0.0232]],

         [[ 0.0148, -0.0101],
          [ 0.0293, -0.0435]]],


        [[[ 0.0422,  0.0521],
          [ 0.0440,  0.0395]],


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


### REFERENCE

https://web.stanford.edu/~nanbhas/blog/forward-hooks-pytorch/


https://blog.paperspace.com/pytorch-hooks-gradient-clipping-debugging/

https://medium.com/the-dl/how-to-use-pytorch-hooks-5041d777f904